In [66]:
!pip install torchsummary
# imports

import torch
from torch import nn
from torchvision.datasets import FashionMNIST
from torchvision import transforms
from torch.utils.data import random_split, DataLoader
from torchsummary import summary
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from tqdm.auto import tqdm
import time
from sklearn.metrics import confusion_matrix, accuracy_score

In [67]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#!pip install torchsummary
# imports

import torch
from torch import nn
from torchvision.datasets import FashionMNIST
from torchvision import transforms
from torch.utils.data import random_split, DataLoader
from torchsummary import summary
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from tqdm.auto import tqdm
import time
from sklearn.metrics import confusion_matrix, accuracy_score
from torchvision.datasets import ImageFolder
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
 tfs = transforms.Compose([
     transforms.Resize((48, 64)),
     transforms.ToTensor()
 ])
 #temp_dataset = ImageFolder("/kaggle/input/bikehorse/Assignment2_BikeHorses", transform=tfs)
 train_dataset = ImageFolder("/kaggle/input/bikeshorses/Assignment2_BikeHorses/Train", transform=tfs)
 val_dataset = ImageFolder("/kaggle/input/bikeshorses/Assignment2_BikeHorses/Test", transform=tfs)
 len(train_dataset), len(val_dataset)

(165, 14)

In [ ]:
# setup device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

In [68]:
# for reproducibility
torch.manual_seed(42)

In [ ]:
# fashionmnist
train_dataset = FashionMNIST("./data", download=True, transform=transforms.ToTensor())
val_dataset = FashionMNIST("./data", download=True, train=False, transform=transforms.ToTensor())
len(train_dataset), len(val_dataset) 

In [ ]:
#Food101
from torch.utils.data import Dataset
from torchvision import datasets, transforms

# Define transformations (adjust for image size)
transform = transforms.Compose([
    transforms.Resize(256),  # Resize images (adjust as needed)
    transforms.CenterCrop(224),  # Crop central area
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # ImageNet normalization
])

# Download and load Food101 dataset
train_dataset = datasets.Food101("./data", download=True, transform=transform)
val_dataset = datasets.Food101("./data", train=False, download=True, transform=transform)

# Print dataset lengths
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")


In [70]:
#SVHN
from torch.utils.data import Dataset
from torchvision import datasets, transforms

# Define transformations (may require adjustments for SVHN)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize pixel values
])

# Download and load SVHN dataset
train_dataset = datasets.SVHN("./data", split='train', download=True, transform=transform)
val_dataset = datasets.SVHN("./data", split='test', download=True, transform=transform)

# Print dataset lengths
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")


Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat
Number of training samples: 73257
Number of validation samples: 26032


In [48]:
train_dataset[0][0].shape

torch.Size([3, 32, 32])

In [71]:
# create dataloaders
batch_size = 128
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [92]:
class Classifier(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding="same"),
            nn.BatchNorm2d(32),
           nn.Sigmoid(),
           nn.MaxPool2d(2)
        )
        #elf.block_1 = nn.Sequential(
    #n.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding="same"),  # Change in_channels to 3
    #n.BatchNorm2d(32),
    #n.ReLU(),
    #n.MaxPool2d(2)
#

        self.block_2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding="same"),
            nn.BatchNorm2d(64),
            nn.Sigmoid(),
            nn.MaxPool2d(2)
        )
        self.block_3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding="same"),
            nn.BatchNorm2d(128),
            nn.Sigmoid(),
            nn.MaxPool2d(2)
        )

        self.block_4 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=2048, out_features=64),
            nn.Sigmoid(),
        )
        self.last_layer = nn.Linear(in_features=64, out_features=10)
        
    def forward(self, x):
        features = self.block_4(
            self.block_3(
                self.block_2(
                    self.block_1(x))))
        activations = self.last_layer(features)
        return (activations, features)

In [93]:
model = Classifier().to(device)
summary(model, (3, 32 , 32))
#summary(model,(3,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             896
       BatchNorm2d-2           [-1, 32, 32, 32]              64
           Sigmoid-3           [-1, 32, 32, 32]               0
         MaxPool2d-4           [-1, 32, 16, 16]               0
            Conv2d-5           [-1, 64, 16, 16]          18,496
       BatchNorm2d-6           [-1, 64, 16, 16]             128
           Sigmoid-7           [-1, 64, 16, 16]               0
         MaxPool2d-8             [-1, 64, 8, 8]               0
            Conv2d-9            [-1, 128, 8, 8]          73,856
      BatchNorm2d-10            [-1, 128, 8, 8]             256
          Sigmoid-11            [-1, 128, 8, 8]               0
        MaxPool2d-12            [-1, 128, 4, 4]               0
          Flatten-13                 [-1, 2048]               0
           Linear-14                   

In [94]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device
              ):
    
    start_time = time.time()

    train_loss = 0    
    model.to(device)
    model.train()
    
    for (X, y) in data_loader:
        # send data to GPU
        X, y = X.to(device), y.to(device)
        # X, y = X.to(device), y.type(torch.LongTensor).to(device)
        
        # 1. forward pass
        y_pred, _ = model(X)

        # 2. calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss
        
        # 3. optimizer zero grad
        optimizer.zero_grad()
        
        # 4. loss backward
        loss.backward()
        
        # 5. optimizer step
        optimizer.step()
    
    train_loss /= len(data_loader)

    end_time = time.time()

    return {"avg_batch_loss": train_loss, "time": (end_time - start_time)* 10**3}

def valid_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               device: torch.device
              ):
    
    # send the model to device
    model.to(device)

    # send the model in eval mode
    model.eval()

    # for confusion matrix and accuracy
    y_true = torch.Tensor([]).to(device)
    y_pred = torch.Tensor([]).to(device)

    with torch.inference_mode(): 
        for X, y in data_loader:
            # Send data to GPU
            X, y = X.to(device), y.to(device)
            
            # 1. Forward pass
            test_pred, _ = model(X)
            
            y_true = torch.cat((y_true, y), dim=0)
            y_pred = torch.cat((y_pred, test_pred.argmax(axis=1)), dim=0)
        
        # send back to cpu
        y_true = y_true.cpu()
        y_pred = y_pred.cpu()

        return {"accuracy": accuracy_score(y_true, y_pred), "confusion_matrix": confusion_matrix(y_true, y_pred, normalize="true")}


In [96]:
# create loss_fn

loss_fn = CrossEntropyLoss()

# create optimizer

lr = 3.2 * (10**-4)
optimizer = Adam(model.parameters(), lr=lr)

In [97]:
epochs = 32

for epoch in tqdm(range(epochs)):
        tres = train_step(model, train_dataloader, loss_fn, optimizer, device)
        print(f"epoch: {epoch}")
        print(f"avg_batch_loss: {tres['avg_batch_loss']}")
        print(f"time: {tres['time']}")   
        print("")

  0%|          | 0/32 [00:00<?, ?it/s]

epoch: 0
avg_batch_loss: 1.6132841110229492
time: 19293.6110496521

epoch: 1
avg_batch_loss: 0.7847603559494019
time: 19639.333963394165

epoch: 2
avg_batch_loss: 0.565780520439148
time: 19340.041160583496

epoch: 3
avg_batch_loss: 0.47616514563560486
time: 19744.540691375732

epoch: 4
avg_batch_loss: 0.42506811022758484
time: 19550.634384155273

epoch: 5
avg_batch_loss: 0.3904785215854645
time: 19656.18848800659

epoch: 6
avg_batch_loss: 0.3644222915172577
time: 19625.080347061157

epoch: 7
avg_batch_loss: 0.3431127071380615
time: 19754.719972610474

epoch: 8
avg_batch_loss: 0.32560887932777405
time: 19266.436338424683

epoch: 9
avg_batch_loss: 0.3104700446128845
time: 19858.958959579468

epoch: 10
avg_batch_loss: 0.2967403829097748
time: 19662.429809570312

epoch: 11
avg_batch_loss: 0.28390929102897644
time: 19377.522945404053

epoch: 12
avg_batch_loss: 0.2723960280418396
time: 19892.36092567444

epoch: 13
avg_batch_loss: 0.2615979313850403
time: 19414.748668670654

epoch: 14
avg_bat

In [98]:
# validation results
vres = valid_step(model, val_dataloader, device)
print(f"accuracy: {vres['accuracy']}")

accuracy: 0.8915949600491703
confusion_matrix: 
[[8.54357798e-01 2.58027523e-02 1.72018349e-03 8.02752294e-03
  2.86697248e-03 5.73394495e-03 4.07110092e-02 4.58715596e-03
  2.23623853e-02 3.38302752e-02]
 [5.09903903e-03 9.55285350e-01 1.96116886e-03 4.51068837e-03
  1.21592469e-02 4.70680526e-03 2.54951951e-03 8.04079231e-03
  2.35340263e-03 3.33398706e-03]
 [1.68715353e-03 1.75946011e-02 8.87924801e-01 2.43432152e-02
  1.15690528e-02 1.25331405e-02 2.89226320e-03 1.27741624e-02
  1.06049651e-02 1.80766450e-02]
 [1.04094379e-03 2.84524636e-02 9.02151284e-03 8.18528799e-01
  5.20471895e-03 6.21096461e-02 6.24566273e-03 4.16377516e-03
  2.98403886e-02 3.53920888e-02]
 [2.77447483e-03 4.24098296e-02 5.54894966e-03 1.03051922e-02
  9.06064209e-01 8.32342449e-03 8.71977804e-03 3.56718193e-03
  3.96353547e-03 8.32342449e-03]
 [1.67785235e-03 8.38926174e-03 3.35570470e-03 2.05536913e-02
  8.38926174e-03 9.27852349e-01 1.59395973e-02 8.38926174e-04
  6.71140940e-03 6.29194631e-03]
 [8.598887

In [99]:
# get features
def get_features(model, data_loader):
    # send the model to device
    model.to(device)

    # send the model in eval mode
    model.eval()

    # for confusion matrix and accuracy
    all_y = torch.Tensor([]).to(device)
    all_x = torch.Tensor([]).to(device)

    with torch.inference_mode(): 
        for X, y in data_loader:
            # Send data to GPU
            X, y = X.to(device), y.to(device)
            
            # 1. Forward pass
            _, features = model(X)
            
            all_y = torch.cat((all_y, y), dim=0)
            all_x = torch.cat((all_x, features), dim=0)
        
        # send back to cpu
        return (all_x.cpu(), all_y.cpu())   

In [100]:
train_x, train_y = get_features(model, train_dataloader)
train_x.shape, train_y.shape

(torch.Size([73257, 64]), torch.Size([73257]))

In [101]:
val_x, val_y = get_features(model, val_dataloader)
val_x.shape, val_y.shape

(torch.Size([26032, 64]), torch.Size([26032]))

In [102]:
# try out sklearn models
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 

fmodel = LogisticRegression(max_iter=1024)
fmodel.fit(train_x, train_y)
print("accuracy:", accuracy_score(val_y, fmodel.predict(val_x)))

accuracy: 0.9018515673017824


In [103]:
# random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score 

fmodel = RandomForestClassifier()
fmodel.fit(train_x, train_y)
print("accuracy:", accuracy_score(val_y, fmodel.predict(val_x)))

accuracy: 0.9031960663798402


In [104]:
# gaussian NB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score 

fmodel = GaussianNB()
fmodel.fit(train_x, train_y)
print("accuracy:", accuracy_score(val_y, fmodel.predict(val_x)))

accuracy: 0.8942839582052858


In [105]:
# multinomial NB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score 

fmodel = MultinomialNB()
fmodel.fit(train_x, train_y)
print("accuracy:", accuracy_score(val_y, fmodel.predict(val_x)))

accuracy: 0.8948985863552551


In [86]:
#SVM
from sklearn.svm import SVC

fmodel = SVC(kernel='linear')
fmodel.fit(train_x, train_y)
print("accuracy:", accuracy_score(val_y, fmodel.predict(val_x)))


KeyboardInterrupt: 

In [36]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

fmodel = GradientBoostingClassifier(n_estimators=50, learning_rate=0.1, max_depth=1)

fmodel.fit(train_x, train_y)

print("accuracy:", accuracy_score(val_y, fmodel.predict(val_x)))


accuracy: 1.0


In [106]:
#SVM
from sklearn.svm import LinearSVC

fmodel = LinearSVC(max_iter=5000)
fmodel.fit(train_x, train_y)
print("accuracy:", accuracy_score(val_y, fmodel.predict(val_x)))

accuracy: 0.9016210817455439


In [107]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Create a KNN classifier with 5 neighbors (you can adjust this later)
knn_model = KNeighborsClassifier(n_neighbors=10)

# Train the model on the training data
knn_model.fit(train_x, train_y)

# Make predictions on the validation data
predictions = knn_model.predict(val_x)

# Evaluate model accuracy
accuracy = accuracy_score(val_y, predictions)
print("accuracy:", accuracy)

accuracy: 0.9058850645359557


In [108]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score

# Create an Extra Trees Classifier instance
etc_model = ExtraTreesClassifier()

# Fit the model to your training data
etc_model.fit(train_x, train_y)

# Make predictions on the validation set
predictions = etc_model.predict(val_x)

# Calculate and print the accuracy
accuracy = accuracy_score(val_y, predictions)
print("accuracy:", accuracy)


accuracy: 0.9029271665642287


In [110]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

# Create a BernoulliNB classifier instance
bnb_model = BernoulliNB()

# Fit the model to your training data
bnb_model.fit(train_x, train_y)

# Make predictions on the validation set
predictions = bnb_model.predict(val_x)

# Calculate and print the accuracy
accuracy = accuracy_score(val_y, predictions)
print("accuracy:", accuracy)

accuracy: 0.1958743085433313


In [111]:
from sklearn.svm import LinearSVC

# Create a LinearSVC classifier instance with L1 penalty
svc_model = LinearSVC(penalty='l1', dual=False)  # Set penalty to 'l1' for L1 regularization

# Fit the model to your training data
svc_model.fit(train_x, train_y)

# Make predictions on the validation set
predictions = svc_model.predict(val_x)

# Calculate and print the accuracy
accuracy = accuracy_score(val_y, predictions)
print("accuracy:", accuracy)


accuracy: 0.9015442532267978


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
